# Desafio 1: Licitações da Secretaria de Educação de São Paulo no Portal de Compras Federal

Neste desafio você irá **acessar o Portal de Compras do Governo Federal (ComprasNET)** e fazer uma busca no sistema para encontrar **todas** as licitações em formato de **pregão eletrônico** em andamento da **Secretaria de Educação de São Paulo**.

Antes de começar o desafio, [acesse o site](http://comprasnet.gov.br/acesso.asp?url=/livre/Pregao/lista_pregao_filtro.asp?Opc=2) e se familiarize com ele. A UASG ([Unidade de Administração de Serviços Gerais](http://www.comprasnet.gov.br/publicacoes/cartilha/glossario.asp)) da Secretaria de Educação de São Paulo é **925013**. É possível encontrar UASG de outras entidades da administração pública na internet facilmente se quiser experimentar :)

Lembre-se que as Ferramentas de Desenvolvimento do Chrome são essenciais nesse desafio!

## Importando as bibliotecas que iremos utilizar

O fluxo da nossa solução será o seguinte:

1. Acessar a página inicial da consulta iniciando uma sessão com `requests_html`
2. Realizar a consulta com os parâmetros apropriados
3. Extrair as informações da tabela de resultados encontrada na página
4. Salvar o conteúdo em CSV usando a biblioteca nativa do Python `csv`
5. Salvar o conteúdo em CSV mas usando a biblioteca `pandas`

Então, os `import` a seguir são os necessários para esta tarefa.

In [1]:
import csv

import pandas as pd
from requests_html import HTMLSession

## Acessando a página inicial

Precisaremos criar uma sessão (como se fosse uma sessão de navegação mesmo) acessando a página inicial para depois fazer a requisição que irá submeter o formulário (realizar a consulta).

Primeiro, vamos guardar a nossa URL inicial em uma variável:

In [2]:
url = "http://comprasnet.gov.br/acesso.asp?url=/livre/Pregao/lista_pregao_filtro.asp?Opc=2"

E então, podemos criar uma sessão e fazer uma requisição para a página inicial:

In [3]:
session = HTMLSession()
first_page = session.get(url)

## Realizando a consulta

Para realizar esta consulta, você deve ter observado que uma requisição `POST` é necessária. Normalmente submissões de formulários são assim mesmo, onde você realiza uma requisição `POST` que o resultado varia a depender dos parâmetros que você insere no _formdata_ (ou dados de formulário). Nesse caso, o _formdata_ deve ser composto pelos parâmetros da nossa consulta (imaginamos).

Qual a URL você identificou nessa requisição que realiza a consulta? Guarde-a numa variável:

In [4]:
form_url = "http://comprasnet.gov.br/livre/Pregao/lista_pregao.asp"

E o __formdata__? Conseguiu identificar os campos? Alguns são sempre necessários e outros podem não ser...

Preencha aqui com os valores adequados (e se desejar, brinque um pouco com esses parâmetros):

In [5]:
formdata = {
    "Opc": "2",
    "rdTpPregao": "E",
    "lstSrp": "T",
    "lstICMS": "T",
    "lstSituacao": "5",
    "lstTipoSuspensao": "0",
    "co_uasg": "925013",
}

Com isso podemos fazer nossa consulta:

In [6]:
form_response = session.post(form_url, data=formdata)

## Agora vamos raspar os itens da página

Analise a página de resultados para encontrar o Xpath que seleciona todas as linhas da tabela de resultados, ou seja, queremos que cada linha da tabela seja um resultado dessa nossa seleção.

Preencha aqui o Xpath correto para que obtenhamos os pregões da tabela de resultados (apenas as linhas que contém pregões mesmo!): 

In [7]:
xpath = "//table[@class='td']//tr[@class='tex3']"
rows = form_response.html.xpath(xpath)

Vamos observar o que a `requests_html` está selecionando... Na célula a seguir, vamos ver como é o HTML do primeiro elemento da lista, ou seja, a primeira linha.

Cada linha deverá ser um elemento `<tr>` com o conteúdo da linha dentro de elementos mais internos. Recomendo analisar um pouco essa estrutura a seguir para entender como a extração de cada campo irá funcionar.

In [8]:
print(rows[0].html)

<tr bgcolor="#FFFFFF" valign="middle" class="tex3">
<td align="center">
<a href="#" onclick="lista_itens('962837','492021');">492021</a>
</td>
<td align="center">925013</td>
<td align="center">PMSP - SECRETARIA MUNICIPAL DE EDUCAÇÃO</td>
<td align="center">19/07/2021 08:00</td>
<td align="center">29/07/2021 09:30</td>
<td align="center">Julgamento/Habilitação</td>
<td align="center">
<a href="javascript:void(0)" onclick="Motivo_Suspensao(962837, '1');">Suspensão Administrativa&#13;
</a>
<br/>
<a href="javascript:void(0)" onclick="Esclar_Pregao(962837);">Esclarecimentos</a><br/>
<a href="javascript:void(0)" onclick="Avisos_Pregao(962837);">Avisos</a><br/>
</td>
</tr>


Vendo como a `requests_html` está enxergando este item, podemos esclarecer uma coisa sobre Xpaths relativos. Se você lembra da aula de Xpath, então talvez se confunda um pouco ao criar os Xpaths para extrair as informações de cada campo da linha.

Mas de onde viria essa confusão? Bem, possivelmente você imaginaria que o elemento atual que a `requests_html` estaria "enxergando" seria o `<tr>` certo? E que para selecionar qualquer elemento a partir deste elemento atual precisaríamos "descer a árvore de elementos" relativamente a partir dele (`"./elemento1/elemento2"`). Porém, a `requests_html` funciona de maneira diferente, e o `<tr>` é um elemento interno do elemento atual (o único descendente imediato, de forma mais específica). Ou seja, é como se tivéssemos um novo documento HTML onde o topo da árvore seria o elemento `<tr>` e apenas ele.

Por isso, aqui você irá preencher o xpath **a partir** do trecho onde está o `tr` nos Xpaths abaixo.

Queremos raspar **o texto** de cada campo apresentado na tabela (exceto o campo "Informações do Pregão") e salvar todos os itens em formato de dicionário numa lista de pregões:

In [9]:
pregoes = []
for row in rows:
    numero_pregao = row.xpath("./tr/td[1]/a/text()")[0]
    codigo_uasg = row.xpath("./tr/td[2]/text()")[0]
    nome_uasg = row.xpath("./tr/td[3]/text()")[0]
    data_hora_inicio = row.xpath("./tr/td[4]/text()")[0]
    data_hora_abertura = row.xpath("./tr/td[5]/text()")[0]
    situacao = row.xpath("./tr/td[6]/text()")[0]
    item = {
        "numero_pregao": numero_pregao,
        "codigo_uasg": codigo_uasg,
        "nome_uasg": nome_uasg,
        "data_hora_inicio": data_hora_inicio,
        "data_hora_abertura": data_hora_abertura,
        "situacao": situacao,
    }
    pregoes.append(item)

## Salvando os itens

Com a nossa lista preenchida, podemos salvar. Primeiro, uma maneira de salvar itens em CSV sem precisar utilizar bibliotecas externas. Esta versão mostra que precisamos de apenas alguns comandos para escrever o CSV:

1. Um comando que indique que queremos criar um arquivo em modo de escrita;
2. Outro que informe quais são os campos que devem ser as colunas do CSV (através dos nomes dos campos do primeiro item da lista de dicionários) neste arquivo
3. Um que escreve o cabeçalho (as colunas) do CSV
4. Um que escreve todas as linhas no arquivo a partir da nossa lista de dicionários

In [10]:
with open('pregoes.csv', 'w') as f:
    fieldnames = pregoes[0].keys()
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(pregoes)

Mas existe maneira mais fácil usando `pandas`, veja se você consegue criar um `DataFrame` a partir da variável `pregoes`:

In [11]:
df_pregoes = pd.DataFrame(pregoes)

E agora salve o `DataFrame` em um arquivo `'pregoes_pandas.csv'` (é recomendável utilizar o parâmetro `index=False` para suprimir a coluna "vazia" do índice do `DataFrame`):

In [12]:
df_pregoes.to_csv('pregoes_pandas.csv', index=False)

## Finalizando o desafio!

Pronto! Temos uma consulta! E ela pode ser modificada para selecionar pregões de outras instituições também!

Se tiver curiosidade e quiser tentar fazer consultas parecidas para outras situações de pregões (que não estão em andamento) [acesse a página de consultas do Portal de Compras do Governo Federal](https://www.gov.br/compras/pt-br/assuntos/consultas-1) e na aba "Pregões" teremos consultas praticamente idênticas a que você acabou de fazer.

Boa jornada!